In [ ]:
# Load the data
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the CSV data into a 2D numpy array
data = np.loadtxt('data/somefile.csv', delimiter=',', skiprows=1, usecols=range(2, 80), dtype=int)

# Truncate and rotate the data 90 degrees counterclockwise
data = data[0:400,:]
data_rotated = np.rot90(data, 1)

print(f"Data shape: {data.shape}")

In [ ]:
# Show the data
plt.figure(figsize=(10, 6))
time_per_sample = 200e-6  # 200 microseconds
frequency_per_band = 1e6  # 1 MHz

# Calculate the extend
num_time_samples = data.shape[0]
num_frequency_bands = data.shape[1]
extent = [0, num_time_samples * time_per_sample, 0, num_frequency_bands * frequency_per_band]

plt.imshow(data_rotated, cmap='hot', aspect='auto', interpolation='none', origin='lower', extent=extent)

print(f"The number of time samples is: {num_time_samples} and the number of frequency bands is: {num_frequency_bands}")

plt.colorbar(label='dB')
plt.title('Spectrogram of Microwave Power 50%')
plt.xlabel('Time (samples)')
plt.ylabel('Frequency (bands)')
plt.show()

In [ ]:
# Analyze frequency variations

# Calculate frequencies
num_frequency_bands = data_rotated.shape[0]
frequencies = np.arange(num_frequency_bands) * frequency_per_band

# Calculate times
num_time_samples = data_rotated.shape[1]
times = np.arange(num_time_samples) * time_per_sample

# Find the frequency index with the maximum power at each time sample
max_power_freq_indices = np.argmax(data_rotated, axis=0)

# Convert frequency indices to actual frequencies
max_power_frequencies = frequencies[max_power_freq_indices]

# Calculate weighted average frequency at each time sample
# Ensure data_rotated is in float for accurate calculations
data_rotated_float = data_rotated.astype(float)

# Calculate the sum of power at each time sample
power_sum = np.sum(data_rotated_float, axis=0)

# Avoid division by zero
power_sum[power_sum == 0] = np.finfo(float).eps

# Calculate weighted sum of frequencies
weighted_freq_sum = np.dot(frequencies, data_rotated_float)
average_frequencies = weighted_freq_sum / power_sum

# Calculate the variance and standard deviation (bandwidth) of frequencies at each time sample
variance_frequencies = np.sum(data_rotated_float * (frequencies[:, np.newaxis] - average_frequencies[np.newaxis, :])**2, axis=0) / power_sum
std_frequencies = np.sqrt(variance_frequencies)

# Plot frequency with maximum power over time
plt.figure(figsize=(15, 6))
plt.plot(times, max_power_frequencies, label='Max Power Frequency', alpha=0.7)
plt.plot(times, average_frequencies, label='Average Frequency', alpha=0.7)
plt.title('Frequency Analysis Over Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.legend()
plt.show()  

In [ ]:
# Overlay the bandwidth on the spectrogram
plt.figure(figsize=(10, 6))
# Plot the spectrogram
plt.imshow(data_rotated, cmap='hot', aspect='auto', interpolation='none', origin='lower', extent=extent)

std_frequencies_mean = np.mean(std_frequencies)

# Overlay the average frequency
plt.plot(times, np.abs(std_frequencies - std_frequencies_mean)*23+std_frequencies_mean, 'g-', linewidth=2, label='Pseudo Bandwidth/Power Estimate')
plt.colorbar(label='dB')
plt.title('Spectrogram with Bandwidth Over Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.legend()
plt.show()

In [ ]:
# Load the data
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max

# Load the CSV data into a 2D numpy array
data = np.loadtxt('data/somefile.csv', delimiter=',', skiprows=1, usecols=range(2, 80), dtype=int)

# Truncate and rotate the data 90 degrees counterclockwise
data = data[0:800,:]
data_rotated = np.rot90(data, 1)

print(f"Data shape: {data.shape}")

# Show the data
plt.figure(figsize=(15, 6))
time_per_sample = 200e-6  # 200 microseconds
frequency_per_band = 1e6  # 1 MHz
# Calculate the extend
num_time_samples = data.shape[0]
num_frequency_bands = data.shape[1]
extent = [0, num_time_samples * time_per_sample, 0, num_frequency_bands * frequency_per_band]

data_rotated_float = data_rotated.astype(float)

# Normalize the data to the range [0, 1]
data_normalized = (data_rotated_float - np.min(data_rotated_float)) / (np.max(data_rotated_float) - np.min(data_rotated_float)) 

# Use peak_local_max to find peaks (outliers) in the data
coordiantes = peak_local_max(data_normalized, min_distance=5, threshold_abs=0.5)

# Extract frequency and time indices of the peaks
frequency_indices = coordiantes[:, 0]
time_indices = coordiantes[:, 1]

# Convert indices to actual time and frequency values
times_peaks = time_indices * time_per_sample
frequencies_peaks = frequency_indices * frequency_per_band

# Plot the spectrogram with outliers highlighted
plt.figure(figsize=(15, 6))
plt.imshow(data_rotated, cmap='hot', aspect='auto', interpolation='none', origin='lower', extent=extent)

#overlay the peaks
plt.scatter(times_peaks, frequencies_peaks, s=50, facecolors='green', edgecolors='green', label='Detected Outliers')

plt.colorbar(label='dB')
plt.title('Spectrogram with Detected Outliers')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.legend()
plt.show()

In [ ]:

# Normalize data to [0, 1]
normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))

# Scale to [0, 255]
scaled_data =(normalized_data * 255).astype(np.uint8)
print(f"the data type is {data.dtype} and the scaled data is {scaled_data.dtype} and the shape is {normalized_data.shape}")


In [ ]:
# Threshold the data to consider only the highest values.
# Here, we'll consider values greater than a chosen threshold. Adjust as needed.
threshold_value = np.percentile(scaled_data, 98)  # let's consider top % of data values
_, threshed_data = cv2.threshold(scaled_data, threshold_value, 255, cv2.THRESH_BINARY)

# Convert data to 8-bit for OpenCV
threshed_data_8bit = np.uint8(threshed_data)

# Find contours
contours, _ = cv2.findContours(threshed_data_8bit, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# get contours of the original data
contour_image = np.zeros_like(scaled_data)
cv2.drawContours(contour_image, contours, -1, (255), 1)  # draw contours

In [ ]:
# Rotate the image for easy plotting
contour_image = np.rot90(contour_image, 1)
data = np.rot90(data, 1)

In [ ]:
# Plot the contour data
plt.figure(figsize=(10, 6))
# Plot the rotated data with zoom on the x-axis (which is now the y-axis of original data)
plt.imshow(contour_image, cmap='gray', aspect='auto', interpolation='none')

# Adjust x-axis to focus on the first 1000 columns
plt.title('Contour of Spectrogram')
plt.xlabel('Time (samples)')
plt.ylabel('Frequency (bands)')
plt.show()

In [ ]:
# Apply  moprhological operations to clean up the image along with masking

# Using a 5x5 kernel to erode the image
#kernel = np.ones((1, 2), np.uint8)
kernel = np.ones((2,1), np.uint8)
eroded_data = cv2.erode(contour_image, kernel, iterations=1)

# Dilate the eroded image
#kernel = np.ones((4, 2), np.uint8)
kernel = np.ones((2, 4), np.uint8)
dilated_data = cv2.dilate(eroded_data, kernel, iterations=3)

#kernel = np.ones((2, 2), np.uint8)
kernel = np.ones((2, 2), np.uint8)
final_data = cv2.erode(dilated_data, kernel, iterations=3)

# Mask the data
final_data[ 0:20,:] = 0
final_data[ 60:,:] = 0


# kernel = np.ones((25,45),np.uint8) # You might need to adjust the kernel size based on your image
# final_data = cv2.morphologyEx(eroded_data, cv2.MORPH_CLOSE, kernel)

In [ ]:
plt.figure(figsize=(10, 6))
# Plot the rotated data with zoom on the x-axis (which is now the y-axis of original data)
plt.imshow(final_data, cmap='gray', aspect='auto', interpolation='none')



# Adjust x-axis to focus on the first 1000 columns
plt.title('Cleaned up Contour of Spectrogram')
plt.xlabel('Time (samples)')
plt.ylabel('Frequency (bands)')
plt.show()

In [ ]:
from skimage.morphology import skeletonize
# Skeletonize the image
# Convert the image to a binary one (0 and 1)
binary_image = final_data > 0
skeleton = skeletonize(binary_image)

# Convert the skeleton back to 8-bit for display
skeleton_8bit = (skeleton * 255).astype(np.uint8)

# Display the original and skeletonized images side by side
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.imshow(final_data, cmap='gray', aspect='auto', interpolation='none')
plt.title('Original Image')
plt.xlabel('Time (samples)')
plt.ylabel('Frequency (bands)')

plt.subplot(1, 2, 2)
plt.imshow(skeleton_8bit, cmap='gray', aspect='auto', interpolation='none')
plt.title('Skeletonized Image')
plt.xlabel('Time (samples)')
plt.ylabel('Frequency (bands)')

plt.tight_layout()
plt.show()

In [ ]:
# Find the contours
contours, _ = cv2.findContours(skeleton_8bit, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
from scipy.interpolate import splprep, splev

# Create a figure and axis
fig, ax = plt.subplots()

# Display the binary image
ax.imshow(final_data, cmap='gray', aspect='auto', interpolation='none')
#contours = contours[1:]
# For each contour (arc in your image)
for contour in contours:
    # Reshape the contour to Nx2 format
    points = contour.squeeze()

    # Skip contours with insufficient points or malformed for spline fitting
    if points.ndim != 2 or points.shape[0] < 3:
        continue

    # Fit spline (B-spline)
    tck, u = splprep(points.T, u=None, s=0.999, k=3,  per=False)

    # Use this tck to generate new spline points
    new_points = splev(u, tck)
    ax.plot(new_points[0],new_points[1], 'r-', linewidth=2)

# Show the plot
plt.title('Contours with b-spline overlay')
plt.xlabel('Time (samples)')
plt.ylabel('Frequency (bands)')
plt.show()

In [ ]:
from scipy.interpolate import splprep, splev

# Create a figure and axis
fig, ax = plt.subplots()

# Display the binary image
ax.imshow(final_data, cmap='gray', aspect='auto', interpolation='none')
#contours = contours[1:]
# For each contour (arc in your image)
for contour in contours:
    # Reshape the contour to Nx2 format
    points = contour.squeeze()
    # Skip contours with insufficient points or malformed for spline fitting
    if points.ndim != 2 or points.shape[0] < 3:
        continue
    # Fit spline (B-spline)
    tck, u = splprep(points.T, u=None, s=0.999, k=3,  per=False)

    # Use this tck to generate new spline points
    new_points = splev(u, tck)
    ax.plot(new_points[0],new_points[1], 'r-', linewidth=2)

# Show the plot
plt.show()

In [ ]:
# Use this tck to generate new spline points
new_points = splev(u, tck)
# Fit the points using a second-order polynomial
coefficients = np.polyfit(new_points[0], new_points[1], 5)
poly = np.poly1d(coefficients)

# Use the polynomial to get the y-values of the quadratic arc for the x values
ys_fit = poly(new_points[0])

# Plot the spline points on a separate plot
plt.figure("Spline Plot")
plt.plot(new_points[0], new_points[1], 'r-', linewidth=2)
plt.plot(new_points[0], ys_fit, 'g-', label="Quadratic Fit")
plt.title("Spline Representation")
plt.show()